In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [3]:
data=pd.read_csv('imbalancedFinal.csv')


In [4]:
filtered_data3 = data[data['Road user'] == 2]
filtered_data3.shape

(11227, 15)

In [5]:
num_bins = 2

bed_days_min = filtered_data3['Severity'].min()
bed_days_max = filtered_data3['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data3['Bed_days_category'] = filtered_data3['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_1616/220732921.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data3['Bed_days_category'] = filtered_data3['Severity'].apply(bin_function)


In [6]:
filtered_data3['Bed_days_category'].value_counts()


Bed_days_category
0    10897
1      330
Name: count, dtype: int64

In [7]:
from xgboost import XGBClassifier


X = filtered_data3[['Calendar year','Month','Cause of injury','Age group','Sex','Counterparty','ABS remoteness area','Road user']]


y = filtered_data3['Bed_days_category']


In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before calibration
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(xgb_model, method='sigmoid', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predictions and probabilities after calibration
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    confusion_matrices_after.append(confusion_matrix(y_test, y_pred_after))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Calibration:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_after)} After Calibration:\n', confusion_matrices_after[-1])

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix before and after calibration
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Confusion Matrix for Fold 1 Before Calibration:
 [[1081    9]
 [   7   26]]
Confusion Matrix for Fold 1 After Calibration:
 [[1086    4]
 [  12   21]]
Confusion Matrix for Fold 2 Before Calibration:
 [[1081    9]
 [   9   24]]
Confusion Matrix for Fold 2 After Calibration:
 [[1084    6]
 [  15   18]]
Confusion Matrix for Fold 3 Before Calibration:
 [[1075   15]
 [  12   21]]
Confusion Matrix for Fold 3 After Calibration:
 [[1085    5]
 [  18   15]]
Confusion Matrix for Fold 4 Before Calibration:
 [[1081    9]
 [   9   24]]
Confusion Matrix for Fold 4 After Calibration:
 [[1083    7]
 [  13   20]]
Confusion Matrix for Fold 5 Before Calibration:
 [[1084    6]
 [   9   24]]
Confusion Matrix for Fold 5 After Calibration:
 [[1085    5]
 [  14   19]]
Confusion Matrix for Fold 6 Before Calibration:
 [[1084    6]
 [  12   21]]
Confusion Matrix for Fold 6 After Calibration:
 [[1085    5]
 [  21   12]]
Confusion Matrix for Fold 7 Before Calibration:
 [[1083    7]
 [  12   21]]
Confusion Matrix f

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from sklearn.linear_model import LogisticRegression
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
confusion_matrices = []

# Choose an oversampling method
#oversampler = SMOTE(random_state=42)
# oversampler = ADASYN(random_state=42)
oversampler = RandomOverSampler(random_state=42)

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    
    # Initialize Logistic Regression model
    lr_model = LogisticRegression(random_state=42, max_iter=1000)
    lr_model.fit(X_train_resampled, y_train_resampled)
    
    y_pred_lr = lr_model.predict(X_test)
    
    accuracy_list.append(accuracy_score(y_test, y_pred_lr))
    precision_list.append(precision_score(y_test, y_pred_lr))
    recall_list.append(recall_score(y_test, y_pred_lr))
    f1_list.append(f1_score(y_test, y_pred_lr))
    
    conf_matrix = confusion_matrix(y_test, y_pred_lr)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0).astype(int)

print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


Confusion Matrix for Fold 1:
 [[771 319]
 [  0  33]]
Confusion Matrix for Fold 2:
 [[766 324]
 [  1  32]]
Confusion Matrix for Fold 3:
 [[753 337]
 [  0  33]]
Confusion Matrix for Fold 4:
 [[775 315]
 [  1  32]]
Confusion Matrix for Fold 5:
 [[792 298]
 [  0  33]]
Confusion Matrix for Fold 6:
 [[757 333]
 [  1  32]]
Confusion Matrix for Fold 7:
 [[751 339]
 [  0  33]]
Confusion Matrix for Fold 8:
 [[769 320]
 [  2  31]]
Confusion Matrix for Fold 9:
 [[761 328]
 [  1  32]]
Confusion Matrix for Fold 10:
 [[741 348]
 [  0  33]]
Mean Accuracy: 0.7090035285546259
Accuracy Std Dev: 0.011960696652015915
Mean Precision: 0.09049467957403035
Precision Std Dev: 0.0036588308436477663
Mean Recall: 0.9818181818181818
Recall Std Dev: 0.020100756305184223
Mean F1-score: 0.16569599181568792
F1-score Std Dev: 0.0062151081554833384
Mean Confusion Matrix:
 [[763 326]
 [  0  32]]


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import numpy as np
from sklearn.linear_model import LogisticRegression
# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
confusion_matrices = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    y_pred_xgb = xgb_model.predict(X_test)
    
    accuracy_list.append(accuracy_score(y_test, y_pred_xgb))
    precision_list.append(precision_score(y_test, y_pred_xgb))
    recall_list.append(recall_score(y_test, y_pred_xgb))
    f1_list.append(f1_score(y_test, y_pred_xgb))
    
    conf_matrix = confusion_matrix(y_test, y_pred_xgb)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Confusion Matrix:\n', mean_conf_matrix)

Confusion Matrix for Fold 1:
 [[1081    9]
 [   7   26]]
Confusion Matrix for Fold 2:
 [[1081    9]
 [   9   24]]
Confusion Matrix for Fold 3:
 [[1075   15]
 [  12   21]]
Confusion Matrix for Fold 4:
 [[1081    9]
 [   9   24]]
Confusion Matrix for Fold 5:
 [[1084    6]
 [   9   24]]
Confusion Matrix for Fold 6:
 [[1084    6]
 [  12   21]]
Confusion Matrix for Fold 7:
 [[1083    7]
 [  12   21]]
Confusion Matrix for Fold 8:
 [[1082    7]
 [  14   19]]
Confusion Matrix for Fold 9:
 [[1079   10]
 [   5   28]]
Confusion Matrix for Fold 10:
 [[1082    7]
 [  11   22]]
Mean Accuracy: 0.9835219832286514
Accuracy Std Dev: 0.0029598584665829256
Mean Precision: 0.733474573420127
Precision Std Dev: 0.05479035746208294
Mean Recall: 0.696969696969697
Recall Std Dev: 0.0778498944161523
Mean F1-score: 0.7120853951472211
F1-score Std Dev: 0.052544987295573845
Mean Confusion Matrix:
 [[1081.2    8.5]
 [  10.    23. ]]


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    lr_model = LogisticRegression( random_state=42)
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_lr_before = lr_model.predict(X_test)
    y_prob_lr_before = lr_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_lr_before))
    precision_list_before.append(precision_score(y_test, y_pred_lr_before))
    recall_list_before.append(recall_score(y_test, y_pred_lr_before))
    f1_list_before.append(f1_score(y_test, y_pred_lr_before))
    entropy_list_before.append(calculate_entropy(y_prob_lr_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_lr_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(lr_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_lr_after = calibrated_model.predict(X_test)
    y_prob_lr_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_lr_after))
    precision_list_after.append(precision_score(y_test, y_pred_lr_after))
    recall_list_after.append(recall_score(y_test, y_pred_lr_after))
    f1_list_after.append(f1_score(y_test, y_pred_lr_after))
    entropy_list_after.append(calculate_entropy(y_prob_lr_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_lr_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of I

--- Before Calibration ---
Mean Accuracy: 0.9706065685401498
Accuracy Std Dev: 1.2001926810963823e-05
Mean Precision: 0.0
Precision Std Dev: 0.0
Mean Recall: 0.0
Recall Std Dev: 0.0
Mean F1-score: 0.0
F1-score Std Dev: 0.0
Mean Entropy: 0.10657941223919906
Entropy Std Dev: 0.0035690562249012397
Mean Confusion Matrix:
 [[1089.7    0. ]
 [  33.     0. ]]
--- After Calibration ---
Mean Accuracy: 0.9706065685401498
Accuracy Std Dev: 1.2001926810963823e-05
Mean Precision: 0.0
Precision Std Dev: 0.0
Mean Recall: 0.0
Recall Std Dev: 0.0
Mean F1-score: 0.0
F1-score Std Dev: 0.0
Mean Entropy: 0.09194853620539904
Entropy Std Dev: 0.0029731538825486462
Mean Confusion Matrix:
 [[1089.7    0. ]
 [  33.     0. ]]


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import numpy as np
from sklearn.linear_model import LogisticRegression
# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
confusion_matrices = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    xgb_model = LogisticRegression(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    y_pred_xgb = xgb_model.predict(X_test)
    
    accuracy_list.append(accuracy_score(y_test, y_pred_xgb))
    precision_list.append(precision_score(y_test, y_pred_xgb))
    recall_list.append(recall_score(y_test, y_pred_xgb))
    f1_list.append(f1_score(y_test, y_pred_xgb))
    
    conf_matrix = confusion_matrix(y_test, y_pred_xgb)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Confusion Matrix:\n', mean_conf_matrix)

/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due t

Confusion Matrix for Fold 1:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 2:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 3:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 4:
 [[1090    0]
 [  33    0]]


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due t

Confusion Matrix for Fold 5:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 6:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 7:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 8:
 [[1089    0]
 [  33    0]]
Confusion Matrix for Fold 9:
 [[1089    0]
 [  33    0]]
Confusion Matrix for Fold 10:
 [[1089    0]
 [  33    0]]
Mean Accuracy: 0.9706065685401498
Accuracy Std Dev: 1.2001926810963823e-05
Mean Precision: 0.0
Precision Std Dev: 0.0
Mean Recall: 0.0
Recall Std Dev: 0.0
Mean F1-score: 0.0
F1-score Std Dev: 0.0
Mean Confusion Matrix:
 [[1089.7    0. ]
 [  33.     0. ]]


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
from xgboost import XGBClassifier


X = filtered_data3[['Calendar year','Month','Cause of injury','Age group','Sex','Counterparty','ABS remoteness area','Road user']]


y = filtered_data3['Bed_days_category']



X_train , X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

xgb_model = XGBClassifier(random_state=42)

xgb_model.fit(X_train, y_train)


y_pred_xgb = xgb_model.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
class_report_xgb = classification_report(y_test, y_pred_xgb)

precision_xgb = precision_score(y_test, y_pred_xgb) 
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
print('Accuracy:', acc_xgb)
print('Confusion Matrix:\n', conf_matrix_xgb)
print('Classification Report:\n', class_report_xgb)
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1_xgb)

Accuracy: 0.98337785693084
Confusion Matrix:
 [[3228   27]
 [  29   85]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      3255
           1       0.76      0.75      0.75       114

    accuracy                           0.98      3369
   macro avg       0.88      0.87      0.87      3369
weighted avg       0.98      0.98      0.98      3369

Precision: 0.7589285714285714
Recall: 0.7456140350877193
F1-score: 0.7522123893805309


In [7]:
model =DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)



acc = accuracy_score(y_test, pred)
conf = confusion_matrix(y_test, pred)
class_report = classification_report(y_test, pred)

precision = precision_score(y_test, pred) 
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('Accuracy:', acc)
print('Confusion Matrix:\n', conf)
print('Classification Report:\n', class_report)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.9783318492134164
Confusion Matrix:
 [[3208   47]
 [  26   88]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      3255
           1       0.65      0.77      0.71       114

    accuracy                           0.98      3369
   macro avg       0.82      0.88      0.85      3369
weighted avg       0.98      0.98      0.98      3369

Precision: 0.6518518518518519
Recall: 0.7719298245614035
F1-score: 0.7068273092369478


In [8]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', random_state=42)

# آموزش مدل با داده‌های آموزشی
mlp_model.fit(X_train
              , y_train)

# Evaluate the model
y_pred_mlp = mlp_model.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
conf_matrix_mlp = confusion_matrix(y_test, y_pred_mlp)
class_report_mlp = classification_report(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print('Accuracy:', acc_mlp)
print('Confusion Matrix:\n', conf_matrix_mlp)
print('Classification Report:\n', class_report_mlp)
print('Precision:', precision_mlp)
print('Recall:', recall_mlp)
print('F1-score:', f1_mlp)

Accuracy: 0.9661620658949243
Confusion Matrix:
 [[3255    0]
 [ 114    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3255
           1       0.00      0.00      0.00       114

    accuracy                           0.97      3369
   macro avg       0.48      0.50      0.49      3369
weighted avg       0.93      0.97      0.95      3369

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anacon

In [9]:
model = SVC(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

acc = accuracy_score(y_test, pred)
conf = confusion_matrix(y_test, pred)
class_report = classification_report(y_test, pred)

precision = precision_score(y_test, pred) 
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

print('Accuracy:', acc)
print('Confusion Matrix:\n', conf)
print('Classification Report:\n', class_report)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.9661620658949243
Confusion Matrix:
 [[3255    0]
 [ 114    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3255
           1       0.00      0.00      0.00       114

    accuracy                           0.97      3369
   macro avg       0.48      0.50      0.49      3369
weighted avg       0.93      0.97      0.95      3369

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anacon

In [10]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# Evaluate the model
y_pred_lr = lr_model.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
class_report_lr = classification_report(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

print('Accuracy:', acc_lr)
print('Confusion Matrix:\n', conf_matrix_lr)
print('Classification Report:\n', class_report_lr)
print('Precision:', precision_lr)
print('Recall:', recall_lr)
print('F1-score:', f1_lr)

Accuracy: 0.9661620658949243
Confusion Matrix:
 [[3255    0]
 [ 114    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3255
           1       0.00      0.00      0.00       114

    accuracy                           0.97      3369
   macro avg       0.48      0.50      0.49      3369
weighted avg       0.93      0.97      0.95      3369

Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anacon

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
uncertainty_list = []  # List to store uncertainty for each fold
confusion_matrices = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and fit Logistic Regression model
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    # Predict probabilities and classes
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    
    # Calculate and store uncertainty as the mean entropy
    # Entropy formula: -sum(p * log2(p)) for each class probability
    entropy = -np.mean(np.sum(y_prob * np.log2(y_prob + 1e-10), axis=1))  # Add a small value to avoid log(0)
    uncertainty_list.append(entropy)
    
    # Store metrics
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    f1_list.append(f1_score(y_test, y_pred))
    
    # Store confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean and standard deviation of uncertainty
mean_uncertainty = np.mean(uncertainty_list)
std_uncertainty = np.std(uncertainty_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

# Print results
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_uncertainty)
print('Entropy Std Dev:', std_uncertainty)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification

Confusion Matrix for Fold 1:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 2:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 3:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 4:
 [[1090    0]
 [  33    0]]


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification

Confusion Matrix for Fold 5:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 6:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 7:
 [[1090    0]
 [  33    0]]
Confusion Matrix for Fold 8:
 [[1089    0]
 [  33    0]]
Confusion Matrix for Fold 9:
 [[1089    0]
 [  33    0]]
Confusion Matrix for Fold 10:
 [[1089    0]
 [  33    0]]
Mean Accuracy: 0.9706065685401498
Accuracy Std Dev: 1.2001926810963823e-05
Mean Precision: 0.0
Precision Std Dev: 0.0
Mean Recall: 0.0
Recall Std Dev: 0.0
Mean F1-score: 0.0
F1-score Std Dev: 0.0
Mean Entropy: 0.15298972076264766
Entropy Std Dev: 0.004054175737588128
Mean Confusion Matrix:
 [[1089.7    0. ]
 [  33.     0. ]]


/Users/amin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
